We aim to use `RADAR` to do differential analysis of m6A peaks. I'm starting from `fastq` raw data to align them using `STAR`; then I will use `bam` files to run `RADAR` and explore the results for downstream analysis.

I needed to re-build `STAR` indices as I've described in my notebook at `genomes/build-genome.ipynb` (see my GitHub repo). 

In [1]:
mkdir <- function (output_dir){
    if (!dir.exists(output_dir)){
        dir.create(output_dir)
        print(output_dir)
        print("created!")
    } else {
        print(output_dir)
        print("already exists!")
    }
}

# Peak calling task - Running `RADAR`

https://scottzijiezhang.github.io/RADARmanual/workflow.html

In [2]:
suppressMessages(suppressWarnings(library (RADAR)))
suppressMessages(suppressWarnings(library (tidyverse)))
suppressMessages(suppressWarnings(library (genomation)))

## 1. load `bam` files and make `MeRIP.RADAR` object
I'll save the object for later usage while it takes long time to generate. 

In [3]:
# read_bed12 <- function(herv_bed_file){
#     bed12_colnames = paste0('V',1:12)
#     names(bed12_colnames) = c(
#         'chromosome','start','end','name','score','strand',
#         'thickStart','thickEnd','itemRgb','blockCount','blockSizes', 'blockStarts'
#     )

#     herv_bed = read.table(herv_bed_file)

#     herv_bed_gr = herv_bed %>% dplyr::rename(all_of(bed12_colnames)) %>% GRanges

#     herv_exons = herv_bed %>% convertBed2Exons

#     herv_exons_grl = split(herv_exons,mcols(herv_exons)$name)

#     return(herv_exons_grl)
# }

In [4]:
run_radar <- function(GTF, bamDir,outputDir,threads){
    mkdir(outputDir)
    
    radar <- countReads(
        samplenames=c('U1','U2','T1','T2'),
        annotation = GTF,
        # gtf = GTF,
        bamFolder=bamDir,
        modification = "m6A", 
        strandToKeep = "opposite",
        threads = threads,
    )
        
    return(radar)
}

In [5]:
# radar_results = list()

### erv

In [ ]:
radar = run_radar(
    GTF = '~/tools/HERVs/files/package-entities-erv.gtf.gz',
    bamDir='align/bam_erv',
    outputDir='radar_herv/erv',
    threads = 25
)

[1] "radar_herv/erv"
[1] "already exists!"
[1] "Stage: index bam file align/bam_erv/U1.input.bam"
[1] "Stage: index bam file align/bam_erv/U1.m6A.bam"
[1] "Stage: index bam file align/bam_erv/U2.input.bam"
[1] "Stage: index bam file align/bam_erv/U2.m6A.bam"
[1] "Stage: index bam file align/bam_erv/T1.input.bam"
[1] "Stage: index bam file align/bam_erv/T1.m6A.bam"
[1] "Stage: index bam file align/bam_erv/T2.input.bam"
[1] "Stage: index bam file align/bam_erv/T2.m6A.bam"
Reading gtf file to obtain gene model
Filter out ambiguous model...


Import genomic features from the file as a GRanges object ... 
OK

Prepare the 'metadata' data frame ... 
OK

Make the TxDb object ... 
OK



In [ ]:
radar <- normalizeLibrary(radar, boxPlot=0)

In [ ]:
radar <- adjustExprLevel(radar)
# radar <- adjustExprLevel(radar, adjustBy = "pos")

# If you expect intensive alternative splicing events cross the experimental groups, 
# using gene-level read counts to represent pre-IP RNA level could leads to bias. 
# Therefore, the user can also choose to use the local bin-level read count to adjust 
# the pre-IP RNA level variation

variable(radar) <- data.frame( group = c(rep("Untreated",2),rep("Treated",2)))

In [ ]:
radar <- filterBins(radar,minCountsCutOff = 0)

radar <- diffIP_parallel(radar,thread = 30)

In [ ]:
saveRDS(radar, 'radar_herv/erv/finalObj.rds')

In [ ]:
radar_all <- reportResult(radar, cutoff = 1.1, Beta_cutoff = 0.00, threads=18) %>% results

In [ ]:
write.table(
    radar_all, 
    file = "radar_herv/erv/result.all.txt",
    row.names=F, sep='\t',quote=FALSE
)

In [ ]:
radar_sig <- reportResult(radar, cutoff = 0.10, Beta_cutoff = 0.5, threads=18) %>% results

In [ ]:
write.table(
    radar_sig, 
    file = "radar_herv/erv/result.sig.txt",
    row.names=F, sep='\t',quote=FALSE
)

### line

In [ ]:
radar = run_radar(
    GTF = '~/tools/HERVs/files/package-entities-line.gtf.gz',
    bamDir='align/bam_line',
    outputDir='radar_herv/line',
    threads = 25
)

[1] "radar_herv/line"
[1] "created!"
[1] "Stage: index bam file align/bam_line/U1.input.bam"
[1] "Stage: index bam file align/bam_line/U1.m6A.bam"
[1] "Stage: index bam file align/bam_line/U2.input.bam"
[1] "Stage: index bam file align/bam_line/U2.m6A.bam"
[1] "Stage: index bam file align/bam_line/T1.input.bam"
[1] "Stage: index bam file align/bam_line/T1.m6A.bam"
[1] "Stage: index bam file align/bam_line/T2.input.bam"
[1] "Stage: index bam file align/bam_line/T2.m6A.bam"
Reading gtf file to obtain gene model
Filter out ambiguous model...


Import genomic features from the file as a GRanges object ... 
OK

Prepare the 'metadata' data frame ... 
OK

Make the TxDb object ... 
OK



In [ ]:
radar <- normalizeLibrary(radar, boxPlot=0)

In [ ]:
radar <- adjustExprLevel(radar)
# radar <- adjustExprLevel(radar, adjustBy = "pos")

# If you expect intensive alternative splicing events cross the experimental groups, 
# using gene-level read counts to represent pre-IP RNA level could leads to bias. 
# Therefore, the user can also choose to use the local bin-level read count to adjust 
# the pre-IP RNA level variation

variable(radar) <- data.frame( group = c(rep("Untreated",2),rep("Treated",2)))

In [ ]:
radar <- filterBins(radar,minCountsCutOff = 0)

radar <- diffIP_parallel(radar,thread = 30)

In [ ]:
saveRDS(radar, 'radar_herv/line/finalObj.rds')

In [ ]:
radar_all <- reportResult(radar, cutoff = 1.1, Beta_cutoff = 0.00, threads=18) %>% results

In [ ]:
write.table(
    radar_all, 
    file = "radar_herv/line/result.all.txt",
    row.names=F, sep='\t',quote=FALSE
)

In [ ]:
radar_sig <- reportResult(radar, cutoff = 0.10, Beta_cutoff = 0.5, threads=18) %>% results

In [ ]:
write.table(
    radar_sig, 
    file = "radar_herv/line/result.sig.txt",
    row.names=F, sep='\t',quote=FALSE
)

### rc

In [42]:
radar = run_radar(
    GTF = '~/tools/HERVs/files/package-entities-rc.gtf.gz',
    bamDir='align/bam_rc',
    outputDir='radar_herv/rc',
    threads = 25
)

[1] "radar_herv/rc"
[1] "created!"
[1] "Stage: index bam file align/bam_rc/U1.input.bam"
[1] "Stage: index bam file align/bam_rc/U1.m6A.bam"
[1] "Stage: index bam file align/bam_rc/U2.input.bam"
[1] "Stage: index bam file align/bam_rc/U2.m6A.bam"
[1] "Stage: index bam file align/bam_rc/T1.input.bam"
[1] "Stage: index bam file align/bam_rc/T1.m6A.bam"
[1] "Stage: index bam file align/bam_rc/T2.input.bam"
[1] "Stage: index bam file align/bam_rc/T2.m6A.bam"
Reading gtf file to obtain gene model
Filter out ambiguous model...


Import genomic features from the file as a GRanges object ... 
OK

Prepare the 'metadata' data frame ... 
OK

Make the TxDb object ... 
OK



Gene model obtained from gtf file...
counting reads for each genes, this step may takes a few hours....
Hyper-thread registered: TRUE 
Using 25 thread(s) to count reads in continuous bins...
Time used to count reads: 0.595678393046061 mins... 


In [43]:
radar <- normalizeLibrary(radar, boxPlot=0)

In [44]:
radar <- adjustExprLevel(radar)
# radar <- adjustExprLevel(radar, adjustBy = "pos")

# If you expect intensive alternative splicing events cross the experimental groups, 
# using gene-level read counts to represent pre-IP RNA level could leads to bias. 
# Therefore, the user can also choose to use the local bin-level read count to adjust 
# the pre-IP RNA level variation

variable(radar) <- data.frame( group = c(rep("Untreated",2),rep("Treated",2)))

Adjusting expression level using Input geneSum read count...


In [45]:
radar <- filterBins(radar,minCountsCutOff = 0)

radar <- diffIP_parallel(radar,thread = 30)

Filtering bins with low read counts...
Bins with average counts lower than  0  in both groups have been removed...
Filtering bins that is enriched in IP experiment...The predictor variable has been converted:
Untreated Untreated   Treated   Treated 
        0         0         1         1 
running PoissonGamma test at single beta mode
Hyper-thread registered: TRUE 
Using 30 thread(s) to run PoissonGamma test...
Time used to run PoissonGamma test: 0.00204348166783651 mins... 



In [46]:
saveRDS(radar, 'radar_herv/rc/finalObj.rds')

In [47]:
radar_all <- reportResult(radar, cutoff = 1.1, Beta_cutoff = 0.00, threads=18) %>% results

Hyper-thread registered: TRUE 
Using 18 thread(s) to report merged report...
Time used to report peaks: 0.00560049613316854 mins... 
When merging neighboring significant bins, logFC was reported as the max logFC among these bins.
p-value of these bins were combined by Fisher's method.
There are 2 reported differential loci at FDR < 1.1 and logFoldChange > 0.


In [48]:
write.table(
    radar_all, 
    file = "radar_herv/rc/result.all.txt",
    row.names=F, sep='\t',quote=FALSE
)

In [50]:
# radar_sig <- reportResult(radar, cutoff = 0.10, Beta_cutoff = 0.5, threads=18) %>% results

In [51]:
# write.table(
#     radar_sig, 
#     file = "radar_herv/rc/result.sig.txt",
#     row.names=F, sep='\t',quote=FALSE
# )

### retroposon

In [30]:
radar = run_radar(
    GTF = '~/tools/HERVs/files/package-entities-retroposon.gtf.gz',
    bamDir='align/bam_retroposon',
    outputDir='radar_herv/retroposon',
    threads = 15
)

[1] "radar_herv/retroposon"
[1] "created!"
[1] "Stage: index bam file align/bam_retroposon/U1.input.bam"
[1] "Stage: index bam file align/bam_retroposon/U1.m6A.bam"
[1] "Stage: index bam file align/bam_retroposon/U2.input.bam"
[1] "Stage: index bam file align/bam_retroposon/U2.m6A.bam"
[1] "Stage: index bam file align/bam_retroposon/T1.input.bam"
[1] "Stage: index bam file align/bam_retroposon/T1.m6A.bam"
[1] "Stage: index bam file align/bam_retroposon/T2.input.bam"
[1] "Stage: index bam file align/bam_retroposon/T2.m6A.bam"
Reading gtf file to obtain gene model
Filter out ambiguous model...


Import genomic features from the file as a GRanges object ... 
OK

Prepare the 'metadata' data frame ... 
OK

Make the TxDb object ... 
OK



Gene model obtained from gtf file...
counting reads for each genes, this step may takes a few hours....
Hyper-thread registered: TRUE 
Using 15 thread(s) to count reads in continuous bins...
Time used to count reads: 6.74785027503967 mins... 


In [31]:
radar <- normalizeLibrary(radar, boxPlot=0)

In [32]:
radar <- adjustExprLevel(radar)
# radar <- adjustExprLevel(radar, adjustBy = "pos")

# If you expect intensive alternative splicing events cross the experimental groups, 
# using gene-level read counts to represent pre-IP RNA level could leads to bias. 
# Therefore, the user can also choose to use the local bin-level read count to adjust 
# the pre-IP RNA level variation

variable(radar) <- data.frame( group = c(rep("Untreated",2),rep("Treated",2)))

Adjusting expression level using Input geneSum read count...


In [33]:
radar <- filterBins(radar,minCountsCutOff = 0)

radar <- diffIP_parallel(radar,thread = 30)

Filtering bins with low read counts...
Bins with average counts lower than  0  in both groups have been removed...
Filtering bins that is enriched in IP experiment...The predictor variable has been converted:
Untreated Untreated   Treated   Treated 
        0         0         1         1 
running PoissonGamma test at single beta mode
Hyper-thread registered: TRUE 
Using 30 thread(s) to run PoissonGamma test...
Time used to run PoissonGamma test: 0.0182193557421366 mins... 



In [34]:
saveRDS(radar, 'radar_herv/retroposon/finalObj.rds')

In [35]:
radar_all <- reportResult(radar, cutoff = 1.1, Beta_cutoff = 0.00, threads=18) %>% results

Hyper-thread registered: TRUE 
Using 18 thread(s) to report merged report...
Time used to report peaks: 0.0793903350830078 mins... 
When merging neighboring significant bins, logFC was reported as the max logFC among these bins.
p-value of these bins were combined by Fisher's method.
There are 508 reported differential loci at FDR < 1.1 and logFoldChange > 0.


In [37]:
write.table(
    radar_all, 
    file = "radar_herv/retroposon/result.all.txt",
    row.names=F, sep='\t',quote=FALSE
)

In [38]:
radar_sig <- reportResult(radar, cutoff = 0.10, Beta_cutoff = 0.5, threads=18) %>% results

Hyper-thread registered: TRUE 
Using 18 thread(s) to report merged report...
Time used to report peaks: 0.0207521875699361 mins... 
When merging neighboring significant bins, logFC was reported as the max logFC among these bins.
p-value of these bins were combined by Fisher's method.
There are 55 reported differential loci at FDR < 0.1 and logFoldChange > 0.5.


In [41]:
write.table(
    radar_sig, 
    file = "radar_herv/retroposon/result.sig.txt",
    row.names=F, sep='\t',quote=FALSE
)

### satellite

In [7]:
radar = run_radar(
    GTF = '~/tools/HERVs/files/package-entities-satellite.gtf.gz',
    bamDir='align/bam_satellite',
    outputDir='radar_herv/satellite',
    threads = 15
)

[1] "radar_herv/satellite"
[1] "created!"
[1] "Stage: index bam file align/bam_satellite/U1.input.bam"
[1] "Stage: index bam file align/bam_satellite/U1.m6A.bam"
[1] "Stage: index bam file align/bam_satellite/U2.input.bam"
[1] "Stage: index bam file align/bam_satellite/U2.m6A.bam"
[1] "Stage: index bam file align/bam_satellite/T1.input.bam"
[1] "Stage: index bam file align/bam_satellite/T1.m6A.bam"
[1] "Stage: index bam file align/bam_satellite/T2.input.bam"
[1] "Stage: index bam file align/bam_satellite/T2.m6A.bam"
Reading gtf file to obtain gene model
Filter out ambiguous model...


Import genomic features from the file as a GRanges object ... 
OK

Prepare the 'metadata' data frame ... 
OK

Make the TxDb object ... 
OK



Gene model obtained from gtf file...
counting reads for each genes, this step may takes a few hours....
Hyper-thread registered: TRUE 
Using 15 thread(s) to count reads in continuous bins...
Time used to count reads: 2.03983365694682 mins... 


In [12]:
radar <- normalizeLibrary(radar, boxPlot=0)

In [13]:
radar <- adjustExprLevel(radar)
# radar <- adjustExprLevel(radar, adjustBy = "pos")

# If you expect intensive alternative splicing events cross the experimental groups, 
# using gene-level read counts to represent pre-IP RNA level could leads to bias. 
# Therefore, the user can also choose to use the local bin-level read count to adjust 
# the pre-IP RNA level variation

variable(radar) <- data.frame( group = c(rep("Untreated",2),rep("Treated",2)))

Adjusting expression level using Input geneSum read count...


In [14]:
radar <- filterBins(radar,minCountsCutOff = 0)

radar <- diffIP_parallel(radar,thread = 30)

Filtering bins with low read counts...
Bins with average counts lower than  0  in both groups have been removed...
Filtering bins that is enriched in IP experiment...The predictor variable has been converted:
Untreated Untreated   Treated   Treated 
        0         0         1         1 
running PoissonGamma test at single beta mode
Hyper-thread registered: TRUE 
Using 30 thread(s) to run PoissonGamma test...
Time used to run PoissonGamma test: 0.00667332013448079 mins... 



In [18]:
saveRDS(radar, 'radar_herv/satellite/finalObj.rds')

In [26]:
radar_all <- reportResult(radar, cutoff = 1.1, Beta_cutoff = 0.00, threads=18) %>% results

Hyper-thread registered: TRUE 
Using 18 thread(s) to report merged report...
Time used to report peaks: 0.0091277281443278 mins... 
When merging neighboring significant bins, logFC was reported as the max logFC among these bins.
p-value of these bins were combined by Fisher's method.
There are 14 reported differential loci at FDR < 1.1 and logFoldChange > 0.


In [29]:
write.table(
    radar_all, 
    file = "radar_herv/satellite/result.all.txt",
    row.names=F, sep='\t',quote=FALSE
) 

### sine

In [ ]:
radar = run_radar(
    GTF = '~/tools/HERVs/files/package-entities-sine.gtf.gz',
    bamDir='align/bam_sine',
    outputDir='radar_herv/sine',
    threads = 35
)

[1] "radar_herv/sine"
[1] "already exists!"
[1] "Stage: index bam file align/bam_sine/U1.input.bam"
[1] "Stage: index bam file align/bam_sine/U1.m6A.bam"
[1] "Stage: index bam file align/bam_sine/U2.input.bam"
[1] "Stage: index bam file align/bam_sine/U2.m6A.bam"
[1] "Stage: index bam file align/bam_sine/T1.input.bam"
[1] "Stage: index bam file align/bam_sine/T1.m6A.bam"
[1] "Stage: index bam file align/bam_sine/T2.input.bam"
[1] "Stage: index bam file align/bam_sine/T2.m6A.bam"
Reading gtf file to obtain gene model
Filter out ambiguous model...


Import genomic features from the file as a GRanges object ... 
OK

Prepare the 'metadata' data frame ... 
OK

Make the TxDb object ... 
OK



In [69]:
radar <- normalizeLibrary(radar, boxPlot=0)

In [70]:
radar <- adjustExprLevel(radar)
# radar <- adjustExprLevel(radar, adjustBy = "pos")

# If you expect intensive alternative splicing events cross the experimental groups, 
# using gene-level read counts to represent pre-IP RNA level could leads to bias. 
# Therefore, the user can also choose to use the local bin-level read count to adjust 
# the pre-IP RNA level variation

variable(radar) <- data.frame( group = c(rep("Untreated",2),rep("Treated",2)))

Adjusting expression level using Input geneSum read count...


In [71]:
radar <- filterBins(radar,minCountsCutOff = 0)

radar <- diffIP_parallel(radar,thread = 30)

Filtering bins with low read counts...
Bins with average counts lower than  0  in both groups have been removed...
Filtering bins that is enriched in IP experiment...The predictor variable has been converted:
Untreated Untreated   Treated   Treated 
        0         0         1         1 
running PoissonGamma test at single beta mode
Hyper-thread registered: TRUE 
Using 30 thread(s) to run PoissonGamma test...
Time used to run PoissonGamma test: 0.178535417715708 mins... 



In [72]:
saveRDS(radar, 'radar_herv/sine/finalObj.rds')

In [75]:
radar_all <- reportResult(radar, cutoff = 1.1, Beta_cutoff = 0.00, threads=18) %>% results

Hyper-thread registered: TRUE 
Using 18 thread(s) to report merged report...
Time used to report peaks: 3.99129880666733 mins... 
When merging neighboring significant bins, logFC was reported as the max logFC among these bins.
p-value of these bins were combined by Fisher's method.
There are 4311 reported differential loci at FDR < 1.1 and logFoldChange > 0.


In [77]:
write.table(
    radar_all, 
    file = "radar_herv/sine/result.all.txt",
    row.names=F, sep='\t',quote=FALSE
)

In [78]:
radar_sig <- reportResult(radar, cutoff = 0.10, Beta_cutoff = 0.5, threads=18) %>% results

Hyper-thread registered: TRUE 
Using 18 thread(s) to report merged report...
Time used to report peaks: 0.340062908331553 mins... 
When merging neighboring significant bins, logFC was reported as the max logFC among these bins.
p-value of these bins were combined by Fisher's method.
There are 184 reported differential loci at FDR < 0.1 and logFoldChange > 0.5.


In [79]:
write.table(
    radar_sig, 
    file = "radar_herv/sine/result.sig.txt",
    row.names=F, sep='\t',quote=FALSE
)

### scrna

In [81]:
radar = run_radar(
    GTF = '~/tools/HERVs/files/package-entities-scrna.gtf.gz',
    bamDir='align/bam_scrna',
    outputDir='radar_herv/scrna',
    threads = 25
)

[1] "radar_herv/scrna"
[1] "already exists!"
[1] "Stage: index bam file align/bam_scrna/U1.input.bam"
[1] "Stage: index bam file align/bam_scrna/U1.m6A.bam"
[1] "Stage: index bam file align/bam_scrna/U2.input.bam"
[1] "Stage: index bam file align/bam_scrna/U2.m6A.bam"
[1] "Stage: index bam file align/bam_scrna/T1.input.bam"
[1] "Stage: index bam file align/bam_scrna/T1.m6A.bam"
[1] "Stage: index bam file align/bam_scrna/T2.input.bam"
[1] "Stage: index bam file align/bam_scrna/T2.m6A.bam"
Reading gtf file to obtain gene model
Filter out ambiguous model...


Import genomic features from the file as a GRanges object ... 
OK

Prepare the 'metadata' data frame ... 
OK

Make the TxDb object ... 
OK



Gene model obtained from gtf file...
counting reads for each genes, this step may takes a few hours....
Hyper-thread registered: TRUE 
Using 25 thread(s) to count reads in continuous bins...
Time used to count reads: 0.239574722448985 mins... 


ERROR: Error in `colnames<-`(`*tmp*`, value = c(paste(samplenames, "input", sep = "-"), : attempt to set 'colnames' on an object with less than two dimensions


### snrna

In [82]:
radar = run_radar(
    GTF = '~/tools/HERVs/files/package-entities-snrna.gtf.gz',
    bamDir='align/bam_snrna',
    outputDir='radar_herv/snrna',
    threads = 25
)

[1] "radar_herv/snrna"
[1] "created!"
[1] "Stage: index bam file align/bam_snrna/U1.input.bam"
[1] "Stage: index bam file align/bam_snrna/U1.m6A.bam"
[1] "Stage: index bam file align/bam_snrna/U2.input.bam"
[1] "Stage: index bam file align/bam_snrna/U2.m6A.bam"
[1] "Stage: index bam file align/bam_snrna/T1.input.bam"
[1] "Stage: index bam file align/bam_snrna/T1.m6A.bam"
[1] "Stage: index bam file align/bam_snrna/T2.input.bam"
[1] "Stage: index bam file align/bam_snrna/T2.m6A.bam"
Reading gtf file to obtain gene model
Filter out ambiguous model...


Import genomic features from the file as a GRanges object ... 
OK

Prepare the 'metadata' data frame ... 
OK

Make the TxDb object ... 
OK



Gene model obtained from gtf file...
counting reads for each genes, this step may takes a few hours....
Hyper-thread registered: TRUE 
Using 25 thread(s) to count reads in continuous bins...
Time used to count reads: 1.39116697311401 mins... 


In [83]:
radar <- normalizeLibrary(radar, boxPlot=0)

In [84]:
radar <- adjustExprLevel(radar)
# radar <- adjustExprLevel(radar, adjustBy = "pos")

# If you expect intensive alternative splicing events cross the experimental groups, 
# using gene-level read counts to represent pre-IP RNA level could leads to bias. 
# Therefore, the user can also choose to use the local bin-level read count to adjust 
# the pre-IP RNA level variation

variable(radar) <- data.frame( group = c(rep("Untreated",2),rep("Treated",2)))

Adjusting expression level using Input geneSum read count...


In [85]:
radar <- filterBins(radar,minCountsCutOff = 0)

radar <- diffIP_parallel(radar,thread = 30)

Filtering bins with low read counts...
Bins with average counts lower than  0  in both groups have been removed...
Filtering bins that is enriched in IP experiment...The predictor variable has been converted:
Untreated Untreated   Treated   Treated 
        0         0         1         1 
running PoissonGamma test at single beta mode
Hyper-thread registered: TRUE 
Using 30 thread(s) to run PoissonGamma test...
Time used to run PoissonGamma test: 0.00926119486490885 mins... 



In [86]:
saveRDS(radar, 'radar_herv/snrna/finalObj.rds')

In [90]:
summary(radar)

MeRIP.RADAR dataset of 4 samples.
Read count quantified in 50-bp consecutive bins on the transcript.
The total read count for Input and IP samples are (Million reads):
       U1   U2   T1   T2
Input 0.2 0.32 0.36 0.28
IP    0.0 0.00 0.00 0.00
Input gene level read count available.
There are 1 predictor variables/covariates. Can access by function variable(MeRIPdata). 


In [88]:
radar_all <- reportResult(radar, cutoff = 1.1, Beta_cutoff = 0.00, threads=18)

ERROR: Error in reportResult(radar, cutoff = 1.1, Beta_cutoff = 0, threads = 18): Need to run diffIP/diffIP_parallel to test for differential methylation before report result! Currently no test statistics available...


In [91]:
# write.table(
#     radar_all, 
#     file = "radar_herv/snrna/result.all.txt",
#     row.names=F, sep='\t',quote=FALSE
# )

### trna

In [92]:
radar = run_radar(
    GTF = '~/tools/HERVs/files/package-entities-trna.gtf.gz',
    bamDir='align/bam_trna',
    outputDir='radar_herv/trna',
    threads = 25
)

[1] "radar_herv/trna"
[1] "created!"
[1] "Stage: index bam file align/bam_trna/U1.input.bam"
[1] "Stage: index bam file align/bam_trna/U1.m6A.bam"
[1] "Stage: index bam file align/bam_trna/U2.input.bam"
[1] "Stage: index bam file align/bam_trna/U2.m6A.bam"
[1] "Stage: index bam file align/bam_trna/T1.input.bam"
[1] "Stage: index bam file align/bam_trna/T1.m6A.bam"
[1] "Stage: index bam file align/bam_trna/T2.input.bam"
[1] "Stage: index bam file align/bam_trna/T2.m6A.bam"
Reading gtf file to obtain gene model
Filter out ambiguous model...


Import genomic features from the file as a GRanges object ... 
OK

Prepare the 'metadata' data frame ... 
OK

Make the TxDb object ... 
OK



Gene model obtained from gtf file...
counting reads for each genes, this step may takes a few hours....
Hyper-thread registered: TRUE 
Using 25 thread(s) to count reads in continuous bins...
Time used to count reads: 0.429320987065633 mins... 


ERROR: Error in `colnames<-`(`*tmp*`, value = c(paste(samplenames, "input", sep = "-"), : attempt to set 'colnames' on an object with less than two dimensions


___
## Save results into files

### Save results into `bed12` format 
`tidyverse` can not be loaded while using RADAR!

In [ ]:
# # save bed 12 format 
# names(result_all)[1] <- paste0("# ", names(result_all)[1])
# write.table(result_all %>% dplyr::select(c(1:12)) %>% filter(grepl('DAC',name)), file = "radar_erv/result.all.bed",row.names=F, sep="\t", quote=FALSE) 

In [ ]:
sessionInfo()

In [ ]:
!date